# Designing Great API's 
(By learning from Kenneth Reitz's `requests`)


* When writing a package (library), providing it with a good API, is almost as important as it's functionality itself. 
* In this talk, we'll be following Kenneth Reitz popular [requests][requests url] (HTTP for Humans) package, and see what makes it so easy and simple to use.
* **Python3** will be used throughout our investigation  

[requests url]: http://docs.python-requests.org/en/master/

## Requests vs. urllib
* By reviewing how `requests` compares to `urllib` in some typical HTTP usage scenarios, we will extract some tips on what makes a good API.
* In some of the cases we will also review `requests` implementation. 

## Tip #1: Explicit (API endpoints) is better than implicit

In [ ]:
import urllib.request
urllib.request.urlopen('http://python.org/')

In [ ]:
import requests
requests.get('http://python.org/')

* Notice how requests is more concise (hence, clear) about what it will do. 
* `urllib` is getting told implicitly to send a GET request since it didn't receive a data argument
* `requests` function name explicitly mark what it will do. 
* Another thing we can see here is that `requests` returns a helpful string with the request status code when examining it (implements `__repr__()`)

#### [Implementation](https://github.com/kennethreitz/requests/blob/master/requests/api.py)

In [ ]:
def request(method, url, **kwargs):
    with sessions.Session() as session:
        return session.request(method=method, url=url, **kwargs)

def get(url, params=None, **kwargs):
    kwargs.setdefault('allow_redirects', True)
    return request('get', url, params=params, **kwargs)

def post(url, data=None, json=None, **kwargs):
    return request('post', url, data=data, json=json, **kwargs)

* All the HTTP verbs follow a similar flow prior to sending, hence there is a the `request()` main flow function.
* Implementing a "helper function" for each verb that calls `request()`, enables the explicitness we are looking for. 

## Tip #2: No need for getters and setters

In [ ]:
import urllib.request
response = urllib.request.urlopen('http://python.org/')
response.getcode()

In [ ]:
import requests
r = requests.get('http://python.org/')
r.status_code

* Accessing an object property as an actual property (and not a method call) makes the code a little clearer. 
* If you come from other OOP language (hmmm... Java), you might be tempted to use getters and setters to allow future changes to the object properties. 
* No need for that in Python, just use the [**`@property`**](http://www.programiz.com/python-programming/property) decorator.

## Tip #3: Easy access to common functionality

In [ ]:
import urllib.parse
import urllib.request
import json

url = 'http://www.httpbin.org/post'
values = {'name' : 'Michael Foord'}

data = urllib.parse.urlencode(values).encode()
response = urllib.request.urlopen(url, data)
body = response.read().decode()
json.loads(body)

In [ ]:
import requests

url = 'http://www.httpbin.org/post'
data = {'name' : 'Michael Foord'}

response = requests.post(url, data=data)
response.json()

* `requests` provides an out-of-the-box experience for the encoding of the data and loading the`json` response while in `urllib`...
* When creating your API think: how will my package be commonly use? What plugs can I add to make that usage easier?

On the same note, `requests` also provides an elegant way to send `json` content:

In [19]:
import requests

url = 'http://www.httpbin.org/post'
data = {'name' : 'Michael Foord'}

response = requests.post(url, json=data)
response.json()

{'args': {},
 'data': '{"name": "Michael Foord"}',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '25',
  'Content-Type': 'application/json',
  'Host': 'www.httpbin.org',
  'User-Agent': 'python-requests/2.10.0'},
 'json': {'name': 'Michael Foord'},
 'origin': '5.29.130.38',
 'url': 'http://www.httpbin.org/post'}

## Tip #4: Prefer Python data types over self-made ones 

In [ ]:
import urllib.request

gh_url = 'https://api.github.com/user'

password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_mgr.add_password(None, gh_url, 'user', 'pswd')
handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

opener = urllib.request.build_opener(handler)
opener.open(gh_url)

In [ ]:
import requests

requests.get('https://api.github.com/user', auth=('user', 'pswd'))

* `requests` uses Python's data structures to pass the authentication information whereas `urllib` demands you create a special class for that. 

#### [Implementation](https://github.com/kennethreitz/requests/blob/fb014560611f6ebb97e7deb03ad8336c3c8f2db1/requests/models.py#L497-L497):

In [ ]:
def prepare_auth(self, auth, url=''):
    """Prepares the given HTTP auth data."""

    # ...

    if auth:
        if isinstance(auth, tuple) and len(auth) == 2:
            # special-case basic HTTP auth
            auth = HTTPBasicAuth(*auth)

* `requests` internally converts the `(user,pass)` tuple to an authentication class.

## Tip #5: Extensions are a package best friend

In [16]:
from requests.auth import AuthBase

class PizzaAuth(AuthBase):
    """Attaches HTTP Pizza Authentication to the given Request object."""
    def __init__(self, username):
        # setup any auth-related data here
        self.username = username

    def __call__(self, r):
        # modify and return the request
        r.headers['X-Pizza'] = self.username
        return r
        
requests.get('http://httpbin.org/get', auth=PizzaAuth('kenneth')).json()

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.10.0',
  'X-Pizza': 'kenneth'},
 'origin': '5.29.130.38',
 'url': 'http://httpbin.org/get'}

* Depends on the domain and your package, there might be some different quirks that the user might need, providing a (documented!) extension mechanism can help the user adjust.
* In this example we create a special authentication class for our request. 

## Tip #6: Let the user choose how to handle errors

In [10]:
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
try:
    response = urlopen('http://www.httpbin.org/geta')
except HTTPError as e:
    if e.code == 404:
        print('Page not found')
else:
    print('All good')

Page not found


In [11]:
import requests
r = requests.get('http://www.httpbin.org/geta')
if r.ok:
    print('All good')
elif r.status_code == requests.codes.not_found: 
    print('Page not found')

Page not found


In [12]:
from requests.exceptions import HTTPError
import requests
r = requests.get('http://www.httpbin.org/posta')
try:
    r.raise_for_status()
except HTTPError as e:
    if e.response.status_code == 404:
        print('Page not found')

Page not found


* Some programmers prefer exceptions, some prefer checks.  
* In some situations a check is much more elegant and sometimes it's the other way around.  
* It's good to let your users to choose what to use when.